In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import torch
import random
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules import dropout
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from os import path
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from collections import defaultdict
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
# import matplotlib.pyplot as plt

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# !pip install tsai

In [5]:
# from tsai.all import *

In [6]:
flag_cuda = torch.cuda.is_available()
torch.manual_seed(42)
np.random.seed(42)
device= None 
if not flag_cuda:
    print('Using CPU')
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print('Using GPU')

Using GPU


# Datasplit generation

In [7]:
# PATH = "dataset/ECE542_sp2022_Project_TerrainRecognition/"  #'/content/drive/MyDrive/ECE542/CompetitiveProject/ECE542_sp2022_Project_TerrainRecognition'
# TRAIN_PATH = PATH +'/TrainingData/'
# TEST_PATH = PATH + "./TestData/"
# val_subject = 6
# # test_subject = 8

S_PATH = "dataset/ECE542_sp2022_Project_TerrainRecognition/"  #'/content/drive/MyDrive/ECE542/CompetitiveProject/ECE542_sp2022_Project_TerrainRecognition'
PATH = S_PATH +'/TrainingData/'
TEST_PATH = S_PATH + "./TestData/"
val_subject = 6


In [8]:
files = []
for filename in os.listdir(PATH):
    files.append(filename)
files = sorted(files, key = lambda x: (int(x.split('_')[1]),int(x.split('_')[2]), x.split('_')[4] ))
files_train = list(filter(lambda x: int(x.split('_')[1]) not in  [val_subject], files))
files_test = list(filter(lambda x: int(x.split('_')[1]) == val_subject, files))
# files_test = list(filter(lambda x: int(x.split('_')[1])== test_subject, files))

In [9]:
files_test

['subject_006_01__x_time.csv',
 'subject_006_01__x.csv',
 'subject_006_01__y_time.csv',
 'subject_006_01__y.csv',
 'subject_006_02__x_time.csv',
 'subject_006_02__x.csv',
 'subject_006_02__y_time.csv',
 'subject_006_02__y.csv',
 'subject_006_03__x_time.csv',
 'subject_006_03__x.csv',
 'subject_006_03__y_time.csv',
 'subject_006_03__y.csv']

In [10]:
files_train_y = files_train[3::4]
# files_train_y

In [11]:
files_y_val = pd.concat([pd.read_csv(PATH + i , header=None) for i in files_train_y])

In [12]:
# continue_loop = True
# threshold = 0.005
# _, files_split_dist = np.unique(files_y_val, return_counts=True)
# files_split_dist = np.divide(files_split_dist, np.max(files_split_dist))
# while continue_loop:
#   files_train_y_split_1 = random.sample(files_train_y, int(len(files_train_y) * 0.8))
#   files_train_y_split_2 = [i for i in files_train_y if i not in files_train_y_split_1]
#   files_split_1_val = pd.concat([pd.read_csv(PATH + i , header=None) for i in files_train_y_split_1])
#   files_split_2_val = pd.concat([pd.read_csv(PATH + i , header=None) for i in files_train_y_split_2])
#   _, files_split_1_dist = np.unique(files_split_1_val, return_counts=True)
#   files_split_1_dist = np.divide(files_split_1_dist, np.max(files_split_1_dist))
#   _, files_split_2_dist = np.unique(files_split_2_val, return_counts=True)
#   files_split_2_dist = np.divide(files_split_2_dist, np.max(files_split_2_dist))
#   if ((files_split_dist[0] - threshold <= files_split_1_dist[0] <= files_split_dist[0] + threshold) and
#       (files_split_dist[1] - threshold <= files_split_1_dist[1] <= files_split_dist[1] + threshold) and
#       (files_split_dist[2] - threshold <= files_split_1_dist[2] <= files_split_dist[2] + threshold) and
#       (files_split_dist[3] - threshold <= files_split_1_dist[3] <= files_split_dist[3] + threshold) and
#       (files_split_dist[0] - threshold <= files_split_2_dist[0] <= files_split_dist[0] + threshold) and
#       (files_split_dist[1] - threshold <= files_split_2_dist[1] <= files_split_dist[1] + threshold) and
#       (files_split_dist[2] - threshold <= files_split_2_dist[2] <= files_split_dist[2] + threshold) and
#       (files_split_dist[3] - threshold <= files_split_2_dist[3] <= files_split_dist[3] + threshold)):
#           print(files_split_1_dist, files_split_2_dist, files_split_dist)
#           break


In [13]:
seed = 136793727
random.seed(seed)
indices = np.random.choice(len(files_train_y), int(len(files_train_y) * 0.8), replace=False)
files_train_y_split_1 = [files_train_y[i] for i in indices]
files_train_y_split_2 = [i for i in files_train_y if i not in files_train_y_split_1]

In [14]:
files_train_y_split_1_names = ["_".join(i.split("__")[: -1]) for i in files_train_y_split_1]
files_train_y_split_2_names = ["_".join(i.split("__")[: -1]) for i in files_train_y_split_2]

In [15]:
training_files = [i for i in files_train if "_".join(i.split("__")[: -1]) in files_train_y_split_1_names]
validation_files = [i for i in files_train if "_".join(i.split("__")[: -1]) in files_train_y_split_2_names]

In [16]:
# training_files.extend(files_test) 

In [17]:
len(training_files), training_files, validation_files

(80,
 ['subject_001_01__x_time.csv',
  'subject_001_01__x.csv',
  'subject_001_01__y_time.csv',
  'subject_001_01__y.csv',
  'subject_001_02__x_time.csv',
  'subject_001_02__x.csv',
  'subject_001_02__y_time.csv',
  'subject_001_02__y.csv',
  'subject_001_03__x_time.csv',
  'subject_001_03__x.csv',
  'subject_001_03__y_time.csv',
  'subject_001_03__y.csv',
  'subject_001_04__x_time.csv',
  'subject_001_04__x.csv',
  'subject_001_04__y_time.csv',
  'subject_001_04__y.csv',
  'subject_001_05__x_time.csv',
  'subject_001_05__x.csv',
  'subject_001_05__y_time.csv',
  'subject_001_05__y.csv',
  'subject_001_06__x_time.csv',
  'subject_001_06__x.csv',
  'subject_001_06__y_time.csv',
  'subject_001_06__y.csv',
  'subject_002_01__x_time.csv',
  'subject_002_01__x.csv',
  'subject_002_01__y_time.csv',
  'subject_002_01__y.csv',
  'subject_002_02__x_time.csv',
  'subject_002_02__x.csv',
  'subject_002_02__y_time.csv',
  'subject_002_02__y.csv',
  'subject_002_04__x_time.csv',
  'subject_002_04__

# Dataset generation and training

In [18]:
def windows(d, w, t):
  r = np.arange(len(d))
  s = r[::t]
  z = list(zip(s, s + w))
  f = '{0[0]}:{0[1]}'.format
  g = lambda t: d.iloc[t[0]:t[1]]
  ranges = list(map(f,z))
  return ranges, pd.concat(map(g, z), keys=map(f, z))

In [19]:
# def make_dataset(files, X, Y, freq_down=True, flag = False):

#     overlap = 30
#     if flag:
#         overlap = 60
#     total_length = 0
#     for i in range(0, len(files), 4):
#         x_time, x, y_time, y = files[i: i + 4]
#         train_df = None
#         if freq_down:
#             #X_features
#             x_time_df = pd.read_csv(TRAIN_PATH + x_time , header=None)
#             x_df = pd.read_csv(TRAIN_PATH + x , header=None)
#             x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
#             x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # down sampled the frequency

#             #Y_labels
#             y_time_df = pd.read_csv(TRAIN_PATH + y_time , header=None)
#             y_df = pd.read_csv(TRAIN_PATH + y , header=None)
#             y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)

#             train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
#             train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
#             total_length += train_df.shape[0]
#             ranges, windows_df = windows(train_df, 60, overlap)
#             for ran in ranges:
#                 l,r = ran.split(':')
#                 df_range = windows_df.iloc[int(l): int(r)]
#                 if int(r) > len(windows_df):
#                     break
#                 y_values = df_range[9].values
#                 x_values = df_range.drop(columns=[9]).values
#                 X.append(x_values)
#                 Y.append(Counter(list(y_values)).most_common(1)[0][0])
#         else:
#             #X_features
#             x_time_df = pd.read_csv(TRAIN_PATH + x_time, header=None)
#             x_time_df.astype('float64')
#             x_df = pd.read_csv(TRAIN_PATH + x, header=None)
#             x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
#             x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
#             x_combined.set_index('timestamp', inplace=True)

#             #Y_labels
#             y_time_df = pd.read_csv(TRAIN_PATH + y_time , header=None)
#             y_time_df.astype('float64')
#             y_df = pd.read_csv(TRAIN_PATH + y, header=None)
#             y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
#             y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
#             y_combined.set_index('timestamp', inplace=True)
#             train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
#             total_length += train_df.shape[0]
#             ranges, windows_df = windows(train_df, 60, overlap)
#             for ran in ranges:
#                 l,r = ran.split(':')
#                 df_range = windows_df.iloc[int(l): int(r)]
#                 if int(r) > len(windows_df):
#                     break
#                 y_values = df_range['1_y'].values
#                 x_values = df_range.drop(columns=['1_y']).values
#                 X.append(x_values)
#                 Y.append(Counter(list(y_values)).most_common(1)[0][0])
#     print(total_length, len(X), len(y))
#     return X, Y

In [20]:
def make_dataset(files, X, Y,  freq_down=True, window_size=60, overlap = 30, is_test = False):

    overlap = overlap
    if is_test:
        overlap = overlap * 2
    total_length = 0
    for i in range(0, len(files), 4):
        x_time, x, y_time, y = files[i: i + 4]
        train_df = None
        if freq_down:
            #X_features
            x_time_df = pd.read_csv(PATH + x_time , header=None)
            x_df = pd.read_csv(PATH + x , header=None)
            x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
            x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # down sampled the frequency

            #Y_labels
            y_time_df = pd.read_csv(PATH + y_time , header=None)
            y_df = pd.read_csv(PATH + y , header=None)
            y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)

            train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
            train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
            total_length += train_df.shape[0]
            ranges, windows_df = windows(train_df, window_size, overlap)
            for ran in ranges:
                l,r = ran.split(':')
                df_range = windows_df.iloc[int(l): int(r)]
                if int(r) > len(windows_df):
                    break
                y_values = df_range[9].values
                x_values = df_range.drop(columns=[9]).values
                X.append(x_values)
                Y.append(Counter(list(y_values)).most_common(1)[0][0])
        else:
            #X_features
            x_time_df = pd.read_csv(PATH + x_time, header=None)
            x_time_df.astype('float64')
            x_df = pd.read_csv(PATH + x, header=None)
            x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
            x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
            x_combined.set_index('timestamp', inplace=True)

            #Y_labels
            y_time_df = pd.read_csv(PATH + y_time , header=None)
            y_time_df.astype('float64')
            y_df = pd.read_csv(PATH + y, header=None)
            y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
            y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
            y_combined.set_index('timestamp', inplace=True)
            train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
            total_length += train_df.shape[0]
            ranges, windows_df = windows(train_df, window_size, overlap)
            for ran in ranges:
                l,r = ran.split(':')
                df_range = windows_df.iloc[int(l): int(r)]
                if int(r) > len(windows_df):
                    break
                y_values = df_range['1_y'].values
                x_values = df_range.drop(columns=['1_y']).values
                X.append(x_values)
                Y.append(Counter(list(y_values)).most_common(1)[0][0])
    print(total_length, len(X), len(Y))
    return X, Y

In [21]:
# def make_dataset_with_norm(files, X, Y, freq_down=True, window_size=60, overlap = 30, is_test = False, per_subject_norm=True, total_y = []):

#     total_length = 0
#     subject_files = defaultdict(list)
    
#     for i in range(1,9):
#       temp_files = list(filter(lambda x: int(x.split('_')[1]) == i, files))
#       subject_files[i].extend(temp_files)  # separating out all the files
    
#     for key, value in subject_files.items():
#       cur_sub_files = value
#       total_train_df = pd.DataFrame()
#       if len(cur_sub_files) == 0: # skip if no files
#         continue
      
#       for i in range(0, len(cur_sub_files), 4):
#           x_time, x, y_time, y = cur_sub_files[i: i + 4]
#           train_df = None
#           if freq_down:
#               #X_features
#               x_time_df = pd.read_csv(TRAIN_PATH + x_time , header=None)
#               x_df = pd.read_csv(TRAIN_PATH + x , header=None)
#               x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
#               x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # Down sampled the frequency

#               #Y_labels
#               y_time_df = pd.read_csv(TRAIN_PATH + y_time , header=None)
#               y_df = pd.read_csv(TRAIN_PATH + y , header=None)
#               y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)

#               train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
#               train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
#               train_df = train_df.rename({2:'acc_x',3:'acc_y',4:'acc_z',5:'gy_x',6:'gy_y',7:'gy_z',9:'target'}, axis='columns')
#               total_length += train_df.shape[0]
#               if is_test:
#                 total_y.extend(list(y_df.values))
#               if total_train_df.empty:
#                 total_train_df = train_df
#               else:
#                 total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)

#           else:
#               #X_features
#               x_time_df = pd.read_csv(TRAIN_PATH + x_time, header=None)
#               x_time_df.astype('float64')
#               x_df = pd.read_csv(TRAIN_PATH + x, header=None)
#               x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
#               x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
#               x_combined.set_index('timestamp', inplace=True)

#               #Y_labels
#               y_time_df = pd.read_csv(TRAIN_PATH + y_time , header=None)
#               y_time_df.astype('float64')
#               y_df = pd.read_csv(TRAIN_PATH + y, header=None)
#               y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
#               y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
#               y_combined.set_index('timestamp', inplace=True)
#               train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
#               train_df = train_df.rename({'1_x':'acc_x',2:'acc_y',3:'acc_z',4:'gy_x',5:'gy_y',6:'gy_z','1_y':'target'}, axis='columns')
#               total_length += train_df.shape[0]

#               if is_test:
#                 total_y.extend(list(y_df.values))
                
#               if total_train_df.empty:
#                 total_train_df = train_df
#               else:
#                 total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)
      

#       if per_subject_norm:   # after collating subject records, apply standard scalar to normalize subject data
#         labels = total_train_df['target']
#         labels = pd.DataFrame({'target':labels.values})
#         temp_df = total_train_df.drop(columns=['target'])
#         scaled_features = StandardScaler().fit_transform(temp_df.values)
#         scaled_features_df = pd.DataFrame(scaled_features, index=temp_df.index, columns=temp_df.columns)
#         # print(key, scaled_features_df.shape, labels.shape)
#         total_train_df = pd.concat([scaled_features_df.reset_index(drop=True),labels.reset_index(drop=True)], axis=1, ignore_index=True)
#         # print(key, total_train_df.shape)
#         total_train_df.columns = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z','target']
     
#       # print(total_train_df.describe())
      
#       ranges, windows_df = windows(total_train_df, window_size, overlap)
#       for ran in ranges:
#           l,r = ran.split(':')
#           df_range = windows_df.iloc[int(l): int(r)]
#           if int(r) > len(windows_df):
#               break
#           y_values = df_range['target'].values
#           x_values = df_range.drop(columns=['target']).values

#           X.append(x_values)
          
#           Y.append(Counter(list(y_values)).most_common(1)[0][0])
      
#     print(total_length, len(X), len(Y), len(total_y))
#     return X, Y, total_y

In [22]:
def make_dataset_with_norm(files, X, Y, freq_down=True, window_size=60, overlap = 30, is_test = False, per_subject_norm=True, total_y = [], per_session_norm=False):

    total_length = 0
    subject_files = defaultdict(list)
    
    for i in range(1,9):
      temp_files = list(filter(lambda x: int(x.split('_')[1]) == i, files))
      subject_files[i].extend(temp_files)  # separating out all the files
    
    for key, value in subject_files.items():
      cur_sub_files = value
      total_train_df = pd.DataFrame()
      if len(cur_sub_files) == 0: # skip if no files
        continue
      
      for i in range(0, len(cur_sub_files), 4):
          x_time, x, y_time, y = cur_sub_files[i: i + 4]
          train_df = None
          if freq_down:
              #X_features
              x_time_df = pd.read_csv(PATH + x_time , header=None)
              x_df = pd.read_csv(PATH + x , header=None)
              x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
              x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # Down sampled the frequency

              #Y_labels
              y_time_df = pd.read_csv(PATH + y_time , header=None)
              y_df = pd.read_csv(PATH + y , header=None)
              y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)

              train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
              train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
              train_df = train_df.rename({2:'acc_x',3:'acc_y',4:'acc_z',5:'gy_x',6:'gy_y',7:'gy_z',9:'target'}, axis='columns')
              total_length += train_df.shape[0]
              if is_test:
                total_y.extend(list(y_df.values))
              if total_train_df.empty:
                total_train_df = train_df
              else:
                total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)

          else:
              #X_features
              x_time_df = pd.read_csv(PATH + x_time, header=None)
              x_time_df.astype('float64')
              x_df = pd.read_csv(PATH + x, header=None)
              x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
              x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
              x_combined.set_index('timestamp', inplace=True)

              #Y_labels
              y_time_df = pd.read_csv(PATH + y_time , header=None)
              y_time_df.astype('float64')
              y_df = pd.read_csv(PATH + y, header=None)
              y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
              y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
              y_combined.set_index('timestamp', inplace=True)
              train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
              train_df = train_df.rename({'1_x':'acc_x',2:'acc_y',3:'acc_z',4:'gy_x',5:'gy_y',6:'gy_z','1_y':'target'}, axis='columns')
              total_length += train_df.shape[0]

              if is_test:
                total_y.extend(list(train_df['target'].values))

              if per_session_norm:   # apply standard scalar to normalize session data
                labels = train_df['target']
                labels = pd.DataFrame({'target':labels.values})
                temp_df = train_df.drop(columns=['target'])
                scaled_features = StandardScaler().fit_transform(temp_df.values)
                scaled_features_df = pd.DataFrame(scaled_features, index=temp_df.index, columns=temp_df.columns)
                # print(key, scaled_features_df.shape, labels.shape)
                train_df = pd.concat([scaled_features_df.reset_index(drop=True),labels.reset_index(drop=True)], axis=1, ignore_index=True)
                # print(key, total_train_df.shape)
                train_df.columns = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z','target']
                
              if total_train_df.empty:
                total_train_df = train_df
              else:
                total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)
      

      if per_subject_norm:   # after collating subject records, apply standard scalar to normalize subject data
        labels = total_train_df['target']
        labels = pd.DataFrame({'target':labels.values})
        temp_df = total_train_df.drop(columns=['target'])
        scaled_features = StandardScaler().fit_transform(temp_df.values)
        scaled_features_df = pd.DataFrame(scaled_features, index=temp_df.index, columns=temp_df.columns)
        # print(key, scaled_features_df.shape, labels.shape)
        total_train_df = pd.concat([scaled_features_df.reset_index(drop=True),labels.reset_index(drop=True)], axis=1, ignore_index=True)
        # print(key, total_train_df.shape)
        total_train_df.columns = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z','target']
     
      # print(total_train_df.describe())
      
      # ranges, windows_df = windows(total_train_df, window_size, overlap)
      # for ran in ranges:
      #     l,r = ran.split(':')
      #     df_range = windows_df.iloc[int(l): int(r)]
      #     if int(r) > len(windows_df):
      #         break
      #     y_values = df_range['target'].values
      #     x_values = df_range.drop(columns=['target']).values

      #     X.append(x_values)
          
      #     Y.append(Counter(list(y_values)).most_common(1)[0][0])
      addl_rows_df = pd.DataFrame(total_train_df[-window_size:])
      total_train_df =  pd.concat([total_train_df, addl_rows_df], ignore_index = True)
      for i in range(len(total_train_df) - window_size):
          df_range = total_train_df.iloc[i:i+window_size]
          
          y_values = df_range['target'].values
          x_values = df_range.drop(columns=['target']).values

          X.append(x_values)
          
          Y.append(Counter(list(y_values)).most_common(1)[0][0])
      
    print(total_length, len(X), len(Y), len(total_y))
    return X, Y, total_y

freq_down boolean flag can be used to flip between 10hz and 40 hz frequency

In [23]:
# train_X, train_y = [], []
# valid_X, valid_y = [], []
# test_X, test_y = [], []

# make_dataset(files_train, train_X, train_y, freq_down=False, flag=True)
# train_X, train_y = np.array(train_X, dtype="float32"), np.array(train_y,dtype="float32")
# make_dataset(files_val, valid_X, valid_y, freq_down=False, flag=True)
# valid_X, valid_y = np.array(valid_X, dtype="float32"),np.array(valid_y, dtype="float32")
# # make_dataset(files_train, test_X, test_y, freq_down=True, flag=True)
# # test_X, test_y = np.array(test_X, dtype="float32"), np.array(test_y, dtype="float32")
# print("Done splitting the data")

In [24]:
# train_X, train_y = [], []
# valid_X, valid_y = [], []
# test_X, test_y, test_total_y_down = [], [], []

# norm_subjectwise = False
# make_dataset(training_files, train_X, train_y, freq_down=False, flag=True)
# # make_dataset_with_norm(training_files, train_X, train_y, freq_down=False, window_size=60, overlap=60, is_test=False, per_subject_norm=norm_subjectwise)
# train_X, train_y = np.array(train_X, dtype="float32"), np.array(train_y,dtype="float32")
# make_dataset(validation_files, valid_X, valid_y, freq_down=False, flag=True)
# # make_dataset_with_norm(validation_files, valid_X, valid_y, freq_down=False, window_size=60, overlap=60, is_test=False, per_subject_norm=norm_subjectwise)
# valid_X, valid_y = np.array(valid_X, dtype="float32"),np.array(valid_y, dtype="float32")
# # make_dataset(files_test, test_X, test_y, freq_down=False, flag=True)
# make_dataset_with_norm(files_test, test_X, test_y, freq_down=False, window_size=60, overlap=60, is_test=True, per_subject_norm=norm_subjectwise, total_y=test_total_y_down)
# test_X, test_y, test_total_y_down = np.array(test_X, dtype="float32"), np.array(test_y, dtype="float32"),np.array(test_total_y_down, dtype="float32") # test_total_y is actual 10hz sample with no windows
# print("Done splitting the data")

In [25]:
train_X, train_y = [], []
valid_X, valid_y = [], []
test_X, test_y, test_total_y = [], [], []
window_size = 30
make_dataset_with_norm(training_files, train_X, train_y, freq_down=False, window_size=window_size, overlap=1, is_test=False, per_subject_norm=False,total_y=[],per_session_norm=True)
train_X, train_y = np.array(train_X, dtype="float32"), np.array(train_y,dtype="float32")
make_dataset_with_norm(validation_files, valid_X, valid_y, freq_down=False, window_size=window_size, overlap=1,is_test=False, per_subject_norm=False,total_y=[],per_session_norm=True)
valid_X, valid_y = np.array(valid_X, dtype="float32"),np.array(valid_y, dtype="float32")
make_dataset_with_norm(files_test, test_X, test_y, freq_down=False, window_size=window_size, overlap=1, is_test=True, per_subject_norm=False, total_y=test_total_y,per_session_norm=True)
test_X, test_y, test_total_y = np.array(test_X, dtype="float32"), np.array(test_y, dtype="float32"),np.array(test_total_y, dtype="float32")  # test_total_y is actual 40hz sample with no windows
test_X_down, test_y_down, test_total_y_down = [], [], []
make_dataset_with_norm(files_test, test_X_down, test_y_down, freq_down=True, window_size=window_size, overlap=1, is_test=True, per_subject_norm=False, total_y=test_total_y_down,per_session_norm=True)
test_X_down, test_y_down, test_total_y_down = np.array([], dtype="float32"), np.array([], dtype="float32"),np.array(test_total_y_down, dtype="float32") # test_total_y is actual 10hz sample with no windows
print("Done splitting the data")

915214 915214 915214 0
273616 273616 273616 0
152816 152816 152816 152816
38204 38204 38204 38204
Done splitting the data


In [26]:
# scaler = StandardScaler()

In [27]:
# train_X = scaler.fit_transform(train_X.reshape(-1, train_X.shape[-1])).reshape(train_X.shape)
# valid_X = scaler.transform(valid_X.reshape(-1, valid_X.shape[-1])).reshape(valid_X.shape)
# test_X = scaler.transform(test_X.reshape(-1, test_X.shape[-1])).reshape(test_X.shape)

In [28]:
train_X = np.array(train_X)
print(train_X.shape)

(915214, 30, 6)


In [29]:
len(train_y)

915214

In [30]:
test_X.shape, valid_X.shape, test_y.shape, valid_y.shape

((152816, 30, 6), (273616, 30, 6), (152816,), (273616,))

In [31]:
weight  = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_y), y = np.array(train_y))
sample_weights = [round(weight[int(i)],2) for i in train_y]
weight = torch.tensor(weight)
weight = weight.cuda()
weight = weight.to(torch.float32)

In [32]:
np.unique(train_y), weight

(array([0., 1., 2., 3.], dtype=float32),
 tensor([0.3341, 5.6071, 4.1853, 1.6968], device='cuda:0'))

In [33]:
# class Net(nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.conv1 = nn.Conv1d(6,16,3)
#     self.conv2 = nn.Conv1d(16,32,3)
#     self.pool1 = nn.MaxPool1d(2,2)
#     self.conv3 = nn.Conv1d(32,64,3)
#     self.conv4 = nn.Conv1d(64,128,3)
#     self.pool2 = nn.MaxPool1d(2,2)
#     self.fc1 = nn.Linear(512, 120)
#     self.fc2 = nn.Linear(120, 64)
#     self.fc3 = nn.Linear(64,4)
#     self.dropout1 = nn.Dropout(0.1)

#   def forward(self, x):
#     x = F.relu(self.conv1(x.permute(0,2,1)))
#     x = self.pool1(F.relu(self.dropout1(self.conv2(x))))
#     x = F.relu(self.conv3(x))
#     x = self.pool2(F.relu(self.dropout1(self.conv4(x))))
#     # print("1",x.shape)
#     # x = x.view(-1,3456)
#     x = torch.flatten(x,1)
#     # print("2",x.shape)
#     x = F.relu(self.dropout1(self.fc1(x)))
#     x = F.relu(self.dropout1(self.fc2(x)))
#     # print("3", x.shape)
#     x = self.fc3(x)
#     return x

# model = Net()
# if flag_cuda:
#   model.cuda()

In [34]:
# class Net2(nn.Module):
#   def __init__(self):
#     super(Net2, self).__init__()
#     self.conv1 = nn.Conv1d(6,16,3)
#     self.conv2 = nn.Conv1d(16,32,3)
#     self.pool1 = nn.AvgPool1d(2,2)
#     self.conv3 = nn.Conv1d(32,64,3)
#     self.conv4 = nn.Conv1d(64,128,3)
#     self.pool2 = nn.MaxPool1d(2,2)
#     self.fc1 = nn.Linear(1536, 120)
#     self.fc2 = nn.Linear(120, 64)
#     self.fc3 = nn.Linear(64,4)
#     self.dropout1 = nn.Dropout(0.1)
#     self.dropout2 = nn.Dropout(0.25)
#     # self.batch_norm = nn.BatchNorm1d()

#   def forward(self, x):
#     x = F.relu(self.conv1(x.permute(0,2,1)))
#     x = self.pool1(self.dropout2(F.relu(self.conv2(x))))
#     # print("1",x.shape)
#     x = F.relu(self.conv3(x))
#     x = self.pool2(self.dropout1(F.relu(self.conv4(x))))
#     x = torch.flatten(x,1)
#     # print("2",x.shape)
#     x = self.dropout1(F.relu(self.fc1(x)))
#     # print("3", x.shape)
#     x = self.dropout1(F.relu(self.fc2(x)))
#     x = self.fc3(x)
#     return x

# model = Net2()
# if flag_cuda:
#   model.cuda()

In [35]:
# class BiLSTM(nn.Module):
#   def __init__(self, window_size=30):
#     super(BiLSTM, self).__init__()
#     self.lstm1 = nn.LSTM(6, window_size, 2,batch_first=True, bidirectional=True)
#     self.lstm2 =  nn.LSTM(window_size * 2, window_size, 2,batch_first=True,bidirectional=True)
#     self.lstm3 =  nn.LSTM(window_size * 2, window_size, 2, batch_first=True,bidirectional=True)
#     self.fc1 = nn.Linear(window_size * 2 * window_size, window_size * 2)
#     self.fc2 = nn.Linear(window_size * 2, 64)
#     self.fc3 = nn.Linear(64, 4)
#     self.dropout1 = nn.Dropout(0.1)
#     self.dropout2 = nn.Dropout(0.25)
#     self.dropout3 = nn.Dropout(0.2)
#     self.batch_norm = nn.BatchNorm1d(window_size * 2)


#   def forward(self, x):
#     x, self.hidden1 = self.lstm1(x)
#     x = self.dropout3(self.batch_norm(F.relu(x).permute(0, 2, 1)))
#     x = self.dropout1(self.batch_norm(F.relu(self.lstm2(x.permute(0, 2, 1))[0]).permute(0,2,1)))
#     # print("1", x.shape)
#     x = self.dropout3(F.relu(self.lstm3(x.permute(0, 2, 1))[0]))
#     # print("3", x.shape)
#     x = torch.flatten(x,1)
#     x = self.dropout1(F.relu(self.fc1(x)))
#     # print("3", x.shape)
#     x = self.dropout1(F.relu(self.fc2(x)))
#     x = self.fc3(x)
#     # x = nn.Softmax(dim=-1)(x)
#     return x

# model = BiLSTM()
# if flag_cuda:
#   model.cuda()


In [188]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SimpleLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = SimpleLSTM(6, 120, 1, 4)
if flag_cuda:
    model.cuda()

In [189]:
# class ConvLSTM(nn.Module):
#   def __init__(self):
#     super(ConvLSTM, self).__init__()
#     self.conv1 = nn.Conv1d(6,16,3)
#     self.lstm1 = nn.LSTM(16,60,2,batch_first=True, bidirectional=True)
#     self.lstm2 =  nn.LSTM(120,60,2,batch_first=True,bidirectional=True)
#     self.lstm3 =  nn.LSTM(120,60,2,batch_first=True,bidirectional=True)
#     self.fc1 = nn.Linear(7200, 120)
#     self.fc2 = nn.Linear(120, 64)
#     self.fc3 = nn.Linear(64,4)
#     self.dropout1 = nn.Dropout(0.1)
#     self.dropout2 = nn.Dropout(0.25)
#     self.dropout3 = nn.Dropout(0.2)
#     self.batch_norm = nn.BatchNorm1d(120)


#   def forward(self, x):
#     x, self.hidden1 = self.lstm1(x)
#     x = self.dropout3(self.batch_norm(F.relu(x).permute(0, 2, 1)))
#     x = self.dropout1(self.batch_norm(F.relu(self.lstm2(x.permute(0, 2, 1))[0]).permute(0,2,1)))
#     # print("1", x.shape)
#     x = self.dropout3(F.relu(self.lstm3(x.permute(0, 2, 1))[0]))
#     # print("3", x.shape)
#     x = torch.flatten(x,1)
#     x = self.dropout1(F.relu(self.fc1(x)))
#     # print("3", x.shape)
#     x = self.dropout1(F.relu(self.fc2(x)))
#     x = self.fc3(x)
#     return x

# model = BiLSTM()
# if flag_cuda:
#   model.cuda()

In [190]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = nn.CrossEntropyLoss(weight=self.alpha)(input, target)  # Cross-entropy loss
        pt = torch.exp(-ce_loss)  # Probability of true class
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss  # Focal loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [191]:
# Specifying the loss function
criterion = nn.CrossEntropyLoss(weight = weight.float())
# criterion = FocalLoss(gamma=2, alpha=weight)

# Specify optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [192]:
# plt.style.use('seaborn')
# plt.figure(figsize=(8,6),dpi=100)
# text = ["standing / walking", "going downstairs","going upstairs", "walking on grass"]
# labels = list(dict(Counter(train_y)).values())
# for bar in range(0,4):
#     plt.bar(text[bar],labels[bar])

In [193]:
# plt.figure(figsize=(8,6),dpi=100)
# plt.style.use('seaborn')
# text = ["standing / walking", "going downstairs","going upstairs", "walking on grass"]
# labels = list(dict(Counter(valid_y)).values())
# for bar in range(0,4):
#     plt.bar(text[bar],labels[bar])

In [194]:
# weighted_sampler = WeightedRandomSampler(
#     weights=sample_weights,
#     num_samples=len(sample_weights),
#     replacement=True
# )

In [195]:
class TerrainDataset(Dataset):
  def __init__(self, data, target):
    self.data = np.array(data)
    self.target = np.array(target)
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    return self.data[index], self.target[index]


In [196]:
batch_size = 120 #60 #120
train_dataset = TerrainDataset(train_X, train_y)
# x_train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, sampler=weighted_sampler, shuffle=False)
x_train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, shuffle=True)
# y_train_loader = torch.utils.data.DataLoader(train_y,batch_size = batch_size, sampler=weighted_sampler)
validation_dataset = TerrainDataset(valid_X, valid_y)
x_validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = batch_size, shuffle=False)
# y_validation_loader = torch.utils.data.DataLoader(valid_y, batch_size = batch_size)

# test_dataset = TerrainDataset(test_X, test_y)
test_dataset = TerrainDataset(test_X, test_y)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle=False)
# y_test_loader = torch.utils.data.DataLoader(test_y, batch_size = batch_size)

In [197]:
# for data, target in x_train_loader:
#   print(list(map(int, list(target))))
#   break

In [198]:
# import matplotlib.pyplot as plt

n_epochs = 10 #25

def trainNet(model,criterion,optimizer,n_epochs,flag_cuda):

  train_losslist = []
  valid_losslist = []
  valid_loss_min = np.Inf # track change in validation loss
  for epoch in range(1, n_epochs + 1):
      # Keeping track of training and validation loss
      train_loss = 0.0
      valid_loss = 0.0
    
    #   model.train()

      for data,target in x_train_loader:
          if flag_cuda:
              data, target = data.cuda(), target.cuda()
          optimizer.zero_grad()
          output = model(data)
          target = target.type(torch.LongTensor) 
          if flag_cuda:
            output, target = output.cuda(), target.cuda()
          loss = criterion(output, target)

          # Backward pass: compute gradient of loss with respect to parameters
          loss.backward()
          # Perform a single optimization step (parameter update)
          optimizer.step()
          # Update training loss
          train_loss += loss.item()*data.size(0)
          
    #   model.eval()

      for data,target in x_validation_loader:
          with torch.no_grad():
            if flag_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            target = target.type(torch.LongTensor) 

            if flag_cuda:
                output, target = output.cuda(), target.cuda()

            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
        
      # Calculating average losses
      train_loss = train_loss/len(train_X)
      valid_loss = valid_loss/len(valid_X)
      train_losslist.append(train_loss)
      valid_losslist.append(valid_loss)
          
      #Printing training/validation statistics 
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))
      
      # Saving model if validation loss has decreased
      if valid_loss < valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
              valid_loss_min,valid_loss))
          torch.save(model.state_dict(), 'model.pt')
          valid_loss_min = valid_loss
        
  return train_losslist, valid_losslist

# Executing the training
train_losslist, valid_losslist = trainNet(model,criterion,optimizer,n_epochs,flag_cuda)

# Loading the best model
model.load_state_dict(torch.load('model.pt'))

# Plotting the learning curves
# plt.plot(range(1, n_epochs + 1), train_losslist, valid_losslist)
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.legend(['Training','Validation'])
# plt.title("Performance of Baseline Model")
# plt.show()

KeyboardInterrupt: 

In [ ]:
classes = [0.,1.,2.,3.]
y_prediction = []

def assessNet(model,criterion, x_test_loader):
  # Tracking test loss and accuracy
  test_loss = 0.0
  class_correct = list(0. for i in range(4))
  class_total = list(0. for i in range(4))

  # Setting model to evaluate
  model.eval()

  # Iterating over batches of test data

#   print(valid_X.shape)

  for data,target in x_test_loader:
      # Obtaining predictions and loss
      if flag_cuda:
          data, target = data.cuda(), target.cuda()
      output = model(data)
      # output = torch.argmax(output,dim = 1)
      target = target.type(torch.LongTensor) 
      if flag_cuda:
        output, target = output.cuda(), target.cuda()
      loss = criterion(output, target)
      test_loss += loss.item()*data.size(0)
      _, pred = torch.max(output, 1)    
      # Comparing predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      # print(pred)
      y_prediction.extend(pred.cpu().numpy())
      correct = np.squeeze(correct_tensor.numpy()) if not flag_cuda else np.squeeze(correct_tensor.cpu().numpy())
      # Calculating test accuracy for each object class
      for i in range(len(output)):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1

  # Computing the average test loss
  test_loss = test_loss/len(valid_X)
  print('Test Loss: {:.6f}\n'.format(test_loss))

  # Computing the class accuracies
  for i in range(4):
      if class_total[i] > 0:
          print('Test Accuracy of %10s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
      else:
          print('Test Accuracy of %10s: N/A (no training examples)' % (classes[i]))

  # Computing the overall accuracy
  print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))


  return

# model.load_state_dict(torch.load(r'C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\BiLSTM_new_splits_2\model_0.41.pt'))
model.load_state_dict(torch.load("model.pt"))
model.eval()
# assessNet(model,criterion, x_validation_loader)
assessNet(model, criterion, test_dataloader)

In [ ]:
print(classification_report(y_true=test_y, y_pred=y_prediction))

In [ ]:
# cm = confusion_matrix(y_true=test_y, y_pred=y_prediction)
# cmd = ConfusionMatrixDisplay(cm, display_labels=["standing / walking", "going downstairs","going upstairs", "walking on grass"])
# cmd.plot()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt     

# ax= plt.subplot()
# sns.heatmap(cm, annot=True, fmt='g', ax=ax)  #annot=True to annotate cells, ftm='g' to disable scientific notation

# # labels, title and ticks
# ax.set_xlabel('Predicted labels')
# ax.set_ylabel('True labels') 
# ax.set_title('Confusion Matrix') 
# ax.xaxis.set_ticklabels(["standing / walking", "going downstairs","going upstairs", "walking on grass"]) 
# ax.yaxis.set_ticklabels(["standing / walking", "going downstairs","going upstairs", "walking on grass"])

In [ ]:
def get_classification(y_pred, y_actual,window_size,down_sampled=True):
  y_pred_total = []
  for i in range(0,len(y_pred),4):
      y_pred_total.extend([Counter(y_pred[i:i+4]).most_common(1)[0][0]])
  # for i in y_pred:
  #   # current_pred = [int(i)] * window_size
  #   if down_sampled:
  #     for i in range(0,len(current_pred),4):
  #       y_pred_total.extend([Counter(current_pred[i:i+4]).most_common(1)[0][0]])
  #   else:
  #     y_pred_total.extend([i]*window_size)

  print(len(y_pred_total), len(y_actual))
  
  len_pred = len(y_pred_total)
  len_actual = len(y_actual)

  # if len_actual > len_pred:
  #   diff = len_actual - len_pred
  #   y_pred_total.extend([y_pred_total[-1]]* diff)

  # elif len_pred > len_actual:
  #   y_pred_total = y_pred_total[:len_actual] 
  
  print(len(y_pred_total), len(y_actual))

  print(classification_report(y_true=y_actual, y_pred=y_pred_total))

get_classification(y_prediction, test_total_y_down, window_size=60, down_sampled=True)

# Sensor data subject wise analysis

In [ ]:
import copy

In [ ]:
subject_stats = {'mean': [], 'min':[], 'max':[]}
actual_stats = {}

for i in range(6):  # 6 columns stats for all the subjects
  actual_stats[i] = copy.deepcopy(subject_stats)

for key, value in subject_files.items():
  local_files = value
  y_labels = []
  train_df = None
  for i in range(0, len(local_files), 4):
          x_time, x, y_time, y = local_files[i: i + 4]
          train_df = None
        
          #X_features
          x_time_df = pd.read_csv(PATH + x_time, header=None)
          x_time_df.astype('float64')
          x_df = pd.read_csv(PATH + x, header=None)
          x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
          x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
          x_combined.set_index('timestamp', inplace=True)

          #Y_labels
          y_time_df = pd.read_csv(PATH + y_time , header=None)
          y_time_df.astype('float64')
          y_df = pd.read_csv(PATH + y, header=None)
          y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
          y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
          y_combined.set_index('timestamp', inplace=True)
          train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
          y_labels.extend(train_df['1_y'].values)
          train_df = train_df.drop(columns=['1_y'])
  
  for index, col in enumerate(train_df.columns[:6]):
    # print(train_df[col].mean())
    actual_stats[index]['mean'].append(train_df[col].mean())
    actual_stats[index]['min'].append(train_df[col].min())
    actual_stats[index]['max'].append(train_df[col].max())

  '''
  print(f"Subject {key} \n", train_df.columns[:6])     # Uncomment to plot the Class distribution for each subject
  print(key, len(y_labels))
  plt.style.use('seaborn')
  plt.figure(figsize=(8,6),dpi=100)
  plt.title(f"Subject {key} class distribution")
  text = ["standing / walking", "going downstairs","going upstairs", "walking on grass"]
  labels = list(dict(Counter(y_labels)).values())
  for bar in range(0,4):
      plt.bar(text[bar],labels[bar])
  '''

In [ ]:
for key, value in actual_stats.items():

  plt.style.use('seaborn')
  plt.figure(figsize=(8,6),dpi=100)
  plt.title(f"Column {key} Mean for each subject")
  text = [f'subject-{i}' for i in range(1,9)]
  stats = value['mean']
  for bar in range(8):
      plt.bar(text[bar],stats[bar])

  plt.style.use('seaborn')
  plt.figure(figsize=(8,6),dpi=100)
  plt.title(f"Column {key} Min for each subject")
  stats = value['min']
  for bar in range(8):
      plt.bar(text[bar],stats[bar])


  plt.style.use('seaborn')
  plt.figure(figsize=(8,6),dpi=100)
  plt.title(f"Column {key} Max for each subject")
  stats = value['max']
  for bar in range(8):
      plt.bar(text[bar],stats[bar])

# Others

In [ ]:
for key, value in subject_files.items():
  local_files = value
  y_labels = []
  for i in range(0, len(local_files), 4):
          x_time, x, y_time, y = local_files[i: i + 4]
          train_df = None
        
          #X_features
          x_time_df = pd.read_csv(PATH + x_time, header=None)
          x_time_df.astype('float64')
          x_df = pd.read_csv(PATH + x, header=None)
          x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
          x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
          x_combined.set_index('timestamp', inplace=True)

          #Y_labels
          y_time_df = pd.read_csv(PATH + y_time , header=None)
          y_time_df.astype('float64')
          y_df = pd.read_csv(PATH + y, header=None)
          y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
          y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
          y_combined.set_index('timestamp', inplace=True)
          train_df = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')  # Merging using the nearest values
          y_labels.extend(train_df['1_y'].values)
  print(key, len(y_labels))
  plt.style.use('seaborn')
  plt.figure(figsize=(8,6),dpi=100)
  plt.title(f"Subject {key} class distribution")
  text = ["standing / walking", "going downstairs","going upstairs", "walking on grass"]
  labels = list(dict(Counter(y_labels)).values())
  for bar in range(0,4):
      plt.bar(text[bar],labels[bar])

In [ ]:
subject_files[1][:10], subject_files[2][:10] 

In [ ]:
subject_files = defaultdict(list)
total_files = []
for filename in os.listdir(PATH):
    total_files.append(filename)
total_files = sorted(total_files, key = lambda x: (int(x.split('_')[1]),int(x.split('_')[2]), x.split('_')[4] ))
for i in range(1,9):
  files = list(filter(lambda x: int(x.split('_')[1]) == i, total_files))
  subject_files[i].extend(files)

Checking the class distribution in each subject

In [ ]:
model = MGRU_FCN(6,4,25)
model.cuda()
# Specifying the loss function
criterion = nn.CrossEntropyLoss(weight = weight.float())

# Specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# class Resnet(nn.Module):
#   def __init__(self):
#     super(Resnet, self).__init__()
#     self.resnet = ResNet(6,4)

#   def forward(self, x):
#     x = self.resnet(x.permute(0,2,1))
#     return x

model = LSTM_FCN(6,4,60)
model.cuda()
# Specifying the loss function
criterion = nn.CrossEntropyLoss(weight = weight.float())

# Specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import seaborn as sns
from scipy import stats
from scipy.signal import find_peaks

# Model sanity check

In [ ]:
model = BiLSTM()
if flag_cuda:
  model.cuda()

In [ ]:
ex_x, ex_y = train_dataset[0]
# train_dataset.__getitem__(0)

In [ ]:
ex_x.shape, ex_y.shape

In [ ]:
pred = model(torch.Tensor(ex_x[np.newaxis, :, :]).cuda())
pred.shape

In [ ]:
criterion(pred, torch.Tensor(ex_y).unsqueeze(0).cuda())

# Discrepency checker

In [ ]:

true_test_files = sorted(os.listdir(TEST_PATH))
target_path = "Trial_2/"

In [ ]:
class TerrainDataset2(Dataset):
  def __init__(self, data):
    self.data = np.array(data)
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    return self.data[index]


def get_pred(y_pred, len_actual, window_size=60):

  y_pred_total = []
  print(len(y_pred))
  for i in y_pred:
    current_pred = [int(i)] * window_size
    
    for i in range(0,len(current_pred),4):
      y_pred_total.extend([Counter(current_pred[i:i+4]).most_common(1)[0][0]])

  len_pred = len(y_pred_total)

  if len_actual > len_pred:
    diff = len_actual - len_pred
    y_pred_total.extend([y_pred_total[-1]]* diff)
  
  return y_pred_total



def make_dataset_with_norm(true_test_files, model1_path, model2_path, window_size=60, overlap = 30, per_subject_norm=True):
  for i in range(0, len(true_test_files), 3):
      X = []
      x_time, x, y_time = true_test_files[i: i + 3]
      train_df = None
      total_train_df = pd.DataFrame()
      

      #X_features
      x_time_df = pd.read_csv(TEST_PATH + x_time, header=None)
      x_time_df.astype('float64')
      x_df = pd.read_csv(TEST_PATH + x, header=None)
      x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
      x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
      x_combined.set_index('timestamp', inplace=True)

      #Y_labels
      y_time_df = pd.read_csv(TEST_PATH + y_time , header=None)
      y_time_df.astype('float64')
      train_df = x_combined.rename({'1':'acc_x',2:'acc_y',3:'acc_z',4:'gy_x',5:'gy_y',6:'gy_z'}, axis='columns')

        
      if total_train_df.empty:
        total_train_df = train_df
      else:
        total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)
        

      if per_subject_norm:   # after collating subject records, apply standard scalar to normalize subject data
        temp_df = total_train_df.copy()
        scaled_features = StandardScaler().fit_transform(temp_df.values)
        total_train_df = pd.DataFrame(scaled_features, index=temp_df.index, columns=temp_df.columns)
        # print(key, scaled_features_df.shape, labels.shape)
        # print(key, total_train_df.shape)
        total_train_df.columns = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
      
      # print(total_train_df.describe())
        
      ranges, windows_df = windows(total_train_df, window_size, overlap)
      for ran in ranges:
          l,r = ran.split(':')
          df_range = windows_df.iloc[int(l): int(r)]
          if int(r) > len(windows_df):
              break
          x_values = df_range.values

          X.append(x_values)
      # print(len(X), np.array(X).shape)

      X = np.array(X, dtype="float32")

      actual_dataset = TerrainDataset2(X)
      actual_dataset_loader = torch.utils.data.DataLoader(actual_dataset, batch_size = len(y_time_df.values), shuffle=False)

      X = torch.from_numpy(X)

      model1 = BiLSTM()
      model1.load_state_dict(torch.load(model1_path))
      for data in actual_dataset_loader:
          pred1 = model1(data)
          _, pred = torch.max(pred1, 1) 
      pred1 = get_pred(pred, len(y_time_df.values), window_size)
      print(len(pred1))


      model2 = BiLSTM()
      model2.load_state_dict(torch.load(model2_path))
      for data in actual_dataset_loader:
          pred2 = model1(data)
          _, pred = torch.max(pred2, 1) 
      pred2 = get_pred(pred, len(y_time_df.values), window_size)
      print(len(pred2))

      print(classification_report(y_true=pred1, y_pred=pred2))

      cm = confusion_matrix(y_true=pred1, y_pred=pred2)
      cmd = ConfusionMatrixDisplay(cm, display_labels=["standing / walking", "going downstairs","going upstairs", "walking on grass"])
      cmd #.plot()

      break

model_path1 = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\BiLSTM_new_splits_2\model_0.41.pt"
model_path2 = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\decay_rate\model_1e7.pt"
make_dataset_with_norm(true_test_files, model1_path=model_path1, model2_path=model_path2, window_size=60, overlap=60, per_subject_norm=True)


# Prediction generation for test files

In [ ]:
S_PATH = "dataset/ECE542_sp2022_Project_TerrainRecognition/"  #'/content/drive/MyDrive/ECE542/CompetitiveProject/ECE542_sp2022_Project_TerrainRecognition'
TEST_PATH = S_PATH + "./TestData/"


In [ ]:
true_test_files = sorted(os.listdir(TEST_PATH))
target_path = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\Trial_2\predictions_0.27"

In [ ]:
class TerrainDataset2(Dataset):
  def __init__(self, data):
    self.data = np.array(data)
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    return self.data[index]


def get_pred(y_pred, window_size=60):

  y_pred_total = []

  for i in range(0,len(y_pred),4):
    y_pred_total.extend([Counter(y_pred[i:i+4]).most_common(1)[0][0]])

  return y_pred_total

In [ ]:
def get_actual_results(true_test_files, model, window_size=60, per_subject_norm=False, per_session_norm=True):
  predictions = []
  for i in range(0, len(true_test_files), 3):
      X = []
      x_time, x, y_time = true_test_files[i: i + 3]
      train_df = None
      total_train_df = pd.DataFrame()
      

      #X_features
      x_time_df = pd.read_csv(TEST_PATH + x_time, header=None)
      x_time_df.astype('float64')
      x_df = pd.read_csv(TEST_PATH + x, header=None)
      x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
      x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
      x_combined.set_index('timestamp', inplace=True)

      #Y_labels
      y_time_df = pd.read_csv(TEST_PATH + y_time , header=None)
      y_time_df.astype('float64')
      train_df = x_combined.rename({'1':'acc_x',2:'acc_y',3:'acc_z',4:'gy_x',5:'gy_y',6:'gy_z'}, axis='columns')

        
      if total_train_df.empty:
        total_train_df = train_df
      else:
        total_train_df = pd.concat([total_train_df, train_df], ignore_index = True)
        

      if per_subject_norm or per_session_norm:   # after collating subject records, apply standard scalar to normalize subject data
        temp_df = total_train_df.copy()
        scaled_features = StandardScaler().fit_transform(temp_df.values)
        total_train_df = pd.DataFrame(scaled_features, index=temp_df.index, columns=temp_df.columns)
        # print(key, scaled_features_df.shape, labels.shape)
        # print(key, total_train_df.shape)
        total_train_df.columns = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
      
      # print(total_train_df.describe())
      print(total_train_df.shape)

      addl_rows_df = pd.DataFrame(total_train_df.iloc[-window_size:])
      total_train_df =  pd.concat([total_train_df, addl_rows_df], ignore_index = True)
      for i in range(len(total_train_df) - window_size):
          df_range = total_train_df.iloc[i:i+window_size]
          x_values = df_range.values
          X.append(x_values)
          
        
      X = np.array(X, dtype="float32")

      actual_dataset = TerrainDataset2(X)
      actual_dataset_loader = torch.utils.data.DataLoader(actual_dataset, batch_size = 1280, shuffle=False)

      X = torch.from_numpy(X)

      total_pred = []

      for data in actual_dataset_loader:
          pred1 = model(data)
          _, pred = torch.max(pred1, 1)
          total_pred.extend(list(map(int,pred)))
      total_pred = get_pred(total_pred, len(y_time_df.values))
      filename = path.join(target_path,(x_time.split('__')[0] + '__y.csv'))
      y_df = pd.DataFrame(total_pred)
      y_df.to_csv(filename, index=False, header=None)


      predictions.append(pred1)


  return predictions


model1 = BiLSTM()
model1.load_state_dict(torch.load(r'C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\Trial_2\model_0.27_focal.pt'))
model1.eval()


predictions = get_actual_results(true_test_files, model1, window_size=60,per_subject_norm=False, per_session_norm=True)
print(len(predictions))

# Discrepancy checker - File

In [ ]:
target_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Results_from_Sai"
# predicted_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\Trial_2\predictions_60"
predicted_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\Trial_2\predictions_0.27"

In [ ]:
target_files_path = [os.path.join(target_files_folder, i) for i in sorted(os.listdir(target_files_folder))]
predicted_files_path = [os.path.join(predicted_files_folder, i) for i in sorted(os.listdir(predicted_files_folder))]
target_files = [pd.read_csv(i, header=None) for i in target_files_path]
predicted_files = [pd.read_csv(i, header=None) for i in predicted_files_path]

In [ ]:
[len(i) for i in predicted_files], [len(i) for i in target_files]

In [ ]:
cms = []
for i, j in zip(target_files, predicted_files):
    target_values = list(i.values)
    predicted_values = list(j.values)
    print(len(target_values), len(predicted_values))
    cms.append(confusion_matrix(y_true=target_values, y_pred=predicted_values))

In [ ]:
cms[0]

In [ ]:
cms[1]

In [ ]:
cms[2]

In [ ]:
cms[3]